In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch

In [3]:
device = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [3]:
import os, io
# os.environ['CUDA_VISIBLE_DEVICES']='2,3,'
import warnings

warnings.filterwarnings("ignore")  # avoid printing out absolute paths

import copy
import statsmodels
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pathlib import Path
import warnings
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
import torch

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss, MAE
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters
from pytorch_forecasting.data.encoders import NaNLabelEncoder
import pytorch_forecasting
from tqdm import tqdm
import multiprocessing as mp

OSError: /opt/conda/lib/python3.10/site-packages/torchtext/lib/libtorchtext.so: undefined symbol: _ZN2at4_ops10select_int4callERKNS_6TensorElN3c106SymIntE

In [ ]:


def categorize_columns(df, threshold=1):
    """
    Categorize columns of a dataframe into continuous and categorical based on a threshold
    for the number of unique values.

    Parameters:
    df: The dataframe to categorize columns from.
    threshold: The threshold for determining categorical columns based on
                     the unique number of values.

    Returns:
    continuous_cols: List of names of continuous columns.
    categorical_cols: List of names of categorical columns.
    """
    exclude_cols = ['time_idx', 'target', 'stock_id', 'date_id']
    continuous_cols = []
    categorical_cols = []

    for col in df.columns:
        if col in exclude_cols:
            continue
        if pd.api.types.is_numeric_dtype(df[col]):
            if len(df[col].unique()) > threshold:
                continuous_cols.append(col)
            else:
                categorical_cols.append(col)
        elif pd.api.types.is_categorical_dtype(df[col]):
            categorical_cols.append(col)

    return continuous_cols, categorical_cols


def load_csv(file_ids,idx):
    print(idx)
    print(file_ids)
    df = []
    for i in tqdm(file_ids):
        df.append(pd.read_csv(f'data/v12/train_intraday_{i}-{i+9}.csv'))
    return {idx: pd.concat(df, ignore_index=True)}


results = []
print("Loading data")
for i in tqdm(range(0,480,10)):
    results.append(pd.read_csv(f'data/v12/train_intraday_{i}-{i+9}.csv'))

# num_cores = 10
# pool = mp.Pool(num_cores)
# file_ids = list(range(0,480,10))

# steps=5
# pool_res=[]
# for i in range(0,len(file_ids),steps):
#     pool_res.append(pool.apply_async(load_csv, args=(file_ids[i:i+steps],i)))

# pool.close()
# pool.join()
# for res in pool_res:
#     results.append(res.get())

results.append(pd.read_csv(f'data/v12/train_intraday_480-480.csv'))
df = pd.concat(results, ignore_index=True)
df['time_idx'] = range(len(df))
df.fillna(1, inplace=True)
#df = df.drop(["Unnamed: 0"], axis=1)
continuous_cols, categorical_cols = categorize_columns(df)
continuous_cols = continuous_cols[1:] # remove the index col
len(continuous_cols)

max_prediction_length = 1
max_encoder_length = 55
training_cutoff = int(df["time_idx"].max() * 0.8)

training = TimeSeriesDataSet(
        df[lambda x: x.time_idx <= training_cutoff],
        time_idx="time_id",
        target="target",
        group_ids=["stock_id"],
        min_encoder_length=max_encoder_length // 2,
        max_encoder_length=max_encoder_length,
        min_prediction_length=1,
        max_prediction_length=max_prediction_length,
        static_reals=[],
        time_varying_known_categoricals=[],
        time_varying_known_reals=['date_id'],
        time_varying_unknown_categoricals=categorical_cols,
        time_varying_unknown_reals=continuous_cols,
        categorical_encoders={
            'stock_id':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
            'seconds_in_bucket':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
            'imbalance_buy_sell_flag':pytorch_forecasting.data.encoders.NaNLabelEncoder(add_nan=True),
        },
        target_normalizer=GroupNormalizer(
            groups=["stock_id"],
        ),
        add_relative_time_idx=True,
        add_target_scales=True,
        add_encoder_length=True,
        allow_missing_timesteps=True,
        #add_nan=True,
        )

# min_prediction_idx=training.index.time.max() + 1,
validation = TimeSeriesDataSet.from_dataset(training, df, predict=True, stop_randomization=True)
batch_size = 128
train_dataloader = training.to_dataloader(
    train=True,
    batch_size=batch_size,
    num_workers=8,
    pin_memory=True,
)
val_dataloader = validation.to_dataloader(
    train=False,
    batch_size=batch_size,
    num_workers=8,
    pin_memory=True,
)
print(training.get_parameters())

pl.seed_everything(42)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=10, verbose=False, mode="min")
lr_logger = LearningRateMonitor()  # log the learning rate
logger = TensorBoardLogger("lightning_logs/v12")  # logging results to a tensorboard

checkpoint_callback = ModelCheckpoint(
    dirpath='lightning_logs/v12/checkpoints',
    filename='{epoch}-{val_loss:.2f}',
    save_top_k=5,
    verbose=True,
    monitor='val_loss',
    mode='min',
)


In [ ]:

trainer = pl.Trainer(
    max_epochs=20,
    gpus=list(range(torch.cuda.device_count())),
    gradient_clip_val=0.1,
    limit_train_batches=10000,
    callbacks=[lr_logger,checkpoint_callback], #early_stop_callback],
    logger=logger,
    accelerator="ddp",
)

tft = TemporalFusionTransformer.from_dataset(
    training,
    learning_rate=5e-4,
    hidden_size=32,
    attention_head_size=8,
    dropout=0.3,
    hidden_continuous_size=128,
    output_size=1,
    loss=MAE(),
    log_interval=10,  # uncomment for learning rate finder and otherwise, e.g. to 10 for logging every 10 batches
    reduce_on_plateau_patience=4,
)
print(f"Number of parameters in network: {tft.size() / 1e3:.1f}k")
tft = tft.to('cuda')

trainer.fit(
    tft,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

trainer.save_checkpoint("last_v12.ckpt")